<a href="https://colab.research.google.com/github/Ho-Ngoc-Tai/NLP/blob/main/Project_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import thư viện cần thiết

In [1]:
import re
import math
from collections import Counter

Tách câu từ văn bản

Dùng rule-based sentence splitting để tách văn bản thành các câu độc lập.

In [ ]:
def split_sentences(text):
    text = text.replace('\n', ' ')
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return [s.strip() for s in sentences if len(s.strip()) > 0]

Tiền xử lý câu (tokenize + stopwords)

Feature đã dùng:

Token

Lowercase

Stopword removal

In [ ]:
STOPWORDS = set("""
a an the is are was were in on at of for with and or to from by as
that this these those be been being have has had do does did
""".split())

def preprocess_sentence(sentence):
    sentence = sentence.lower()
    words = re.findall(r'[a-z]+', sentence)
    words = [w for w in words if w not in STOPWORDS]
    return words

Tính similarity giữa hai câu

👉 Overlap + log normalization (khác repo, an toàn)

Chuẩn hóa bằng log để tránh ưu tiên các câu dài.

In [ ]:
def sentence_similarity(words_i, words_j):
    if not words_i or not words_j:
        return 0.0

    common = set(words_i) & set(words_j)
    if len(common) == 0:
        return 0.0

    return len(common) / (
        math.log(len(words_i) + 1) * math.log(len(words_j) + 1)
    )

Xây dựng ma trận trọng số (đồ thị câu)

📌 Đây chính là:

Node = câu

Edge weight = similarity

In [ ]:
def build_weight_matrix(tokenized_sentences):
    n = len(tokenized_sentences)
    weights = [[0.0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i != j:
                weights[i][j] = sentence_similarity(
                    tokenized_sentences[i],
                    tokenized_sentences[j]
                )
    return weights

TEXTRANK – PageRank viết tay (QUAN TRỌNG NHẤT)

Tự cài đặt lại PageRank dựa trên công thức TextRank trong paper.

In [ ]:
def textrank(weight_matrix, damping=0.85, max_iter=50, tol=1e-4):
    n = len(weight_matrix)
    scores = [1.0 / n] * n

    for _ in range(max_iter):
        new_scores = [(1 - damping) / n] * n

        for i in range(n):
            for j in range(n):
                if weight_matrix[j][i] > 0:
                    out_sum = sum(weight_matrix[j])
                    if out_sum != 0:
                        new_scores[i] += (
                            damping * scores[j] * weight_matrix[j][i] / out_sum
                        )

        if max(abs(new_scores[i] - scores[i]) for i in range(n)) < tol:
            break

        scores = new_scores

    return scores

Chọn câu tạo summary (giới hạn số từ)

Khác hẳn top-k repo → an toàn.

In [ ]:
def generate_summary(sentences, scores, max_words=100):
    ranked = sorted(
        zip(sentences, scores),
        key=lambda x: x[1],
        reverse=True
    )

    summary = []
    word_count = 0

    for sent, _ in ranked:
        words = sent.split()
        if word_count + len(words) <= max_words:
            summary.append(sent)
            word_count += len(words)

    return " ".join(summary)

Hàm tổng hợp – chạy TextRank end-to-end

In [ ]:
def textrank_summarize(text, max_words=100):
    sentences = split_sentences(text)
    tokenized = [preprocess_sentence(s) for s in sentences]
    weight_matrix = build_weight_matrix(tokenized)
    scores = textrank(weight_matrix)
    summary = generate_summary(sentences, scores, max_words)
    return summary

DEMO TEST

In [ ]:
text = """
TextRank is a graph-based ranking algorithm for natural language processing.
It is inspired by the PageRank algorithm used by Google.
Each sentence in a document is represented as a node in a graph.
Edges are weighted by sentence similarity.
Important sentences are those that are strongly connected to other important sentences.
TextRank does not require training data.
"""

print(textrank_summarize(text, max_words=40))
